In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
from numpy import loadtxt
import tensorflow_addons as tfa

In [3]:
up = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/wo36.csv', low_memory=False, header=None)
down = pd.read_csv('/home/jordan/Documents/DataSets/DimensionData/go22.csv', low_memory=False, header=None)

In [24]:
model1=tf.keras.models.load_model('/home/jordan/Documents/Data-Science/Medium/model.h5')
#model2=load_model('/home/jordan/Documents/Data-Science/Medium/tuned_model.h5')
#model3=load_model('/home/jordan/Documents/Data-Science/Medium/tuned_model2.h5')
model4=tf.keras.models.load_model('/home/jordan/Documents/Data-Science/Medium/tuned_model3.h5')

In [10]:
def snip(data):
    time= data.iloc[:,0]
    time=time.map(lambda x: str(x[6:]))
    time=time.map(lambda x: float(x))

    interface= data.iloc[:,-14]
    interface=interface.map(lambda x: str(x))
    interface=interface.map(lambda x: str(x[15:]))
    interface=interface.map(lambda x: str(x[:-1]))

    jitter=data.iloc[:, -13]
    jitter=jitter.map(lambda x: str(x))
    jitter=jitter.map(lambda x: str(x[8:]))
    jitter=jitter.map(lambda x: str(x[:-1]))
    jitter=jitter.map(lambda x: float(x))

    latency= data.iloc[:,-12]
    latency=latency.map(lambda x: str(x))
    latency=latency.map(lambda x: str(x[9:]))
    latency=latency.map(lambda x: str(x[:-1]))
    latency=latency.map(lambda x: float(x))

    inband= data.iloc[:,-15]
    inband=inband.map(lambda x: str(x))
    inband=inband.map(lambda x: str(x[22:]))
    inband=inband.map(lambda x: str(x[:-5]))
    inband=inband.map(lambda x: float(x))

    outband= data.iloc[:, -5]
    outband=outband.map(lambda x: str(x))
    outband=outband.map(lambda x: x.split('"')[1])
    outband=outband.map(lambda x: str(x[:-4]))
    outband=outband.map(lambda x: float(x))

    packet=data.iloc[:,-4]
    packet=packet.map(lambda x: str(x))
    packet=packet.map(lambda x: str(x[12:]))
    packet=packet.map(lambda x: str(x[:-2]))
    packet=packet.map(lambda x: float(x))

    status=data.iloc[:,-2]
    status=status.map(lambda x: str(x))
    status=status.map(lambda x: str(x[8:]))
    status=status.map(lambda x: str(x[:-1]))

    status[:] = np.where(status=="up", 1,0)
    status=status.map(lambda x: float(x))
    #status=status.rolling(20).mean()
    #status=status.map(np.floor)

    train_init=pd.concat([time, interface, inband, outband, latency, packet, jitter, status], axis=1)
    train_init.columns=['Time', 'Interface','Inband', 'Outband', 'Latency', 'Packet', 'Jitter', 'Status']

    #Dedupp based on interfaces
    train=train_init[train_init.Interface != 'NI2A-WWW1']
    train=train[train.Interface != 'NI2B-WWW1']
    train=train[train.Interface != 'NI2A-WWW2']
    train=train[train.Interface != 'NI2B-WWW2']
    train=train[train.Interface != 'NI2A-WWW3']    
    train=train[train.Interface != 'NI2B-WWW3']
    train=train[train.Interface != 'NI2A-MPLS']
    train['Interface']= train['Interface'].astype('category')
    train['Interface']=train['Interface'].cat.codes

    train_label=train.filter(['Status'])
    train=train.drop(['Status', 'Inband', 'Outband'], axis=1)
    train=train.reset_index(drop=True)

    return train, train_label

In [27]:
up,up_label=snip(up)
up=up.iloc[:-700, :]
up=up.to_numpy()
up_label=up_label.iloc[700:]

In [28]:
down,down_label=snip(down)
down=down.iloc[:-700, :]
down=down.to_numpy()
down_label2=down_label.iloc[700:]

In [29]:
score = model4.evaluate(test, test_label, verbose=0)
print("%s: %.2f%%" % (model4.metrics_names[1], score[1]*100))

ValueError: in user code:

    File "/home/jordan/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1525, in test_function  *
        return step_function(self, iterator)
    File "/home/jordan/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1514, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/jordan/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1507, in run_step  **
        outputs = model.test_step(data)
    File "/home/jordan/.local/lib/python3.9/site-packages/keras/engine/training.py", line 1471, in test_step
        y_pred = self(x, training=False)
    File "/home/jordan/.local/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/jordan/.local/lib/python3.9/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        raise ValueErro

In [ ]:
score = model4.evaluate(test2, test_label2, verbose=0)
print("%s: %.2f%%" % (model4.metrics_names[1], score[1]*100))

In [22]:
model4.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 200)               2000      
                                                                 
 dropout_3 (Dropout)         (None, 200)               0         
                                                                 
 dense_7 (Dense)             (None, 16)                3216      
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                                 
 dense_8 (Dense)             (None, 152)               2584      
                                                                 
 dropout_5 (Dropout)         (None, 152)               0         
                                                                 
Total params: 7,800
Trainable params: 7,800
Non-traina